In [1]:
import asyncio
from InlineAgent.agent import InlineAgent
from InlineAgent.tools import MCPHttp
from IPython.display import clear_output
from IPython.display import display
import ipywidgets as widgets

![Bastion DNS](./static/bastion.png)

Copy the _McpEksStack.BastionInstancePublicDnsName_ CDK stack output to the cell below

In [2]:
# Replace with your EC2 instance IP or hostname
EC2_HOST = "ec2-35-85-57-47.us-west-2.compute.amazonaws.com"
MCP_PORT = 8000

In [3]:
async def setup_agent():
    # Configure connection to your Kubernetes MCP server
    mcp_client = await MCPHttp.create(
        url=f"http://{EC2_HOST}:{MCP_PORT}/sse",
        # You can add headers like authorization headers if needed
        headers={},
        timeout=10,  # Connection timeout
        sse_read_timeout=300  # Read timeout (5 minutes)
    )

    # Create the InlineAgent with the MCP client
    agent = InlineAgent(
        foundation_model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
        instruction="""You are a Kubernetes cluster management assistant that helps users manage their EKS cluster.
You have access to various kubectl commands through an MCP server.
When users ask you about Kubernetes resources or want to perform actions, use the appropriate tools.
Always show the relevant information clearly and explain what you're doing.
""",
        agent_name="kubernetes-assistant",
        action_groups=[
            {
                "name": "KubernetesActions",
                "description": "Tools for managing Kubernetes clusters",
                "mcp_clients": [mcp_client]
            }
        ]
    )

    return agent

In [4]:
output = widgets.Output()
text_input = widgets.Text(
    value='',
    placeholder='Type your question here...',
    description='Query:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)
send_button = widgets.Button(
    description='Send',
    disabled=False,
    button_style='primary',
    tooltip='Send query to Kubernetes assistant',
    icon='paper-plane'
)
clear_button = widgets.Button(
    description='Clear Chat',
    disabled=False,
    button_style='warning',
    tooltip='Clear chat history',
    icon='trash'
)

In [5]:
agent = None

async def initialize():
    global agent
    with output:
        print("Initializing Kubernetes assistant...")
        agent = await setup_agent()
        initial_response = await agent.invoke(
            "Hello! I'm your Kubernetes assistant. How can I help you with your EKS cluster today?"
        )
        print("Assistant: ", initial_response)

In [6]:
# Handle button clicks
def on_send_button_clicked(b):
    user_query = text_input.value
    if not user_query.strip():
        return

    text_input.value = ''

    with output:
        print(f"You: {user_query}")

    # Use asyncio to handle the async agent.invoke call
    async def process_query():
        global agent
        if agent is None:
            agent = await setup_agent()

        response = await agent.invoke(user_query)
        with output:
            print(f"Assistant: {response}")

    asyncio.create_task(process_query())

def on_clear_button_clicked(b):
    with output:
        clear_output()
        print("Chat history cleared.")

In [9]:
# Connect button clicks to handlers
send_button.on_click(on_send_button_clicked)
clear_button.on_click(on_clear_button_clicked)
# Display the UI
from ipywidgets import VBox, HBox, Text, Button, Output, Layout
from IPython.display import display

ui = VBox([
    output,
    HBox([text_input, send_button, clear_button])
])
display(ui)
# Initialize the agent when this cell is run
await initialize()

ImportError: cannot import name 'display_widget' from 'IPython.display' (/home/ec2-user/mcp-eks/.venv/lib/python3.12/site-packages/IPython/display.py)